## Named entity recognition

In [1]:
import spacy

In [2]:
nlp = spacy.load('pt_core_news_lg')

# Frase
doc = nlp(u'Rio de Janeiro é a capital turística do Brasil.')

In [3]:
# Print das Entidades e suas classes
print ("\n-------Exemplo 1 ------\n")
for ent in doc.ents:
    print(ent.label_, ent.text)


-------Exemplo 1 ------

LOC Rio de Janeiro
LOC Brasil


In [4]:
# Frase
doc1 = nlp(u'Enquanto isso na França, Christine Lagarde discutiu os esforços de estímulo de curto prazo em uma '
           u'entrevista recente às 17:00 com o Wall Street Journal')

# Print das Entidades e suas classes
print ("\n-------Exemplo 2 ------\n")
for ent1 in doc1.ents:
    print(ent1.label_, ent1.text)


-------Exemplo 2 ------

LOC França
PER Christine Lagarde
MISC Wall Street Journal


In [10]:
doc = nlp("Enquanto isso na França, Christine Lagarde discutiu os esforços de estímulo de curto prazo em uma entrevista recente às 17:00 com o Wall Street Journal")

for token in doc:
    print(token.text, token.lemma_, token.pos_)

Enquanto enquanto ADP
isso isso PRON
na em o ADP
França França PROPN
, , PUNCT
Christine Christine PROPN
Lagarde Lagarde PROPN
discutiu discutir VERB
os o DET
esforços esforço NOUN
de de ADP
estímulo estímulo NOUN
de de ADP
curto curto ADJ
prazo prazo NOUN
em em ADP
uma um DET
entrevista entrevista NOUN
recente recente ADJ
às a o ADP
17:00 17:00 NOUN
com com ADP
o o DET
Wall Wall PROPN
Street Street PROPN
Journal Journal PROPN
